In [1]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown
import textwrap
import os
from langchain_core.tools import Tool

In [2]:
# using v2rayn
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTP_PROXYS"] = "http://127.0.0.1:10809"


# AI model
model = "gemini-1.5-flash"

In [3]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
chat = ChatGoogleGenerativeAI(model=model,
                              google_api_key=os.getenv("GOOGLE_API_KEY"))


search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERP_API_KEY"))
search_tool = Tool(
    name="serpapi",
    description="A tool that uses the SerpAPI to search the web.",
    func=search.run,
)

tools = [search_tool]


chat_with_tools = chat.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            "You are a helpful assistant. Answer all questions to the best of your ability. When I want to search the web, I will ask you to use the SerpAPI tool."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat_with_tools


chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)